In [1]:
!pip install geopandas

     |████████████████████████████████| 1.0MB 11.8MB/s 
     |████████████████████████████████| 6.5MB 16.6MB/s 
     |████████████████████████████████| 14.8MB 302kB/s 


In [1]:
#Importing libraries
import pandas as pd
import geopandas as gpd
import numpy as np
from geopy.geocoders import Nominatim

In [2]:
#Importing crime data and opening the weather data file:
!wget https://data.buffalony.gov/api/views/d6g9-xbgu/rows.csv?accessType=DOWNLOAD;
#Reading the crime data into a pandas dataframe :
crime_df = pd.read_csv('/content/rows.csv?accessType=DOWNLOAD')
#converting the incident_datetime column into datetime format:
crime_df['incident_datetime']=pd.to_datetime(crime_df['incident_datetime'])
# Sorting the data by time :
crime_df.sort_values('incident_datetime',inplace=True)
# Creating a new neighborhood column with empty values :
crime_df['neighborhood']=np.nan
crime_df['police_dt']=np.nan
crime_df['council_dt']=np.nan

--2021-03-03 13:07:50--  https://data.buffalony.gov/api/views/d6g9-xbgu/rows.csv?accessType=DOWNLOAD
Resolving data.buffalony.gov (data.buffalony.gov)... 52.206.140.205, 52.206.140.199, 52.206.68.26
Connecting to data.buffalony.gov (data.buffalony.gov)|52.206.140.205|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/csv]
Saving to: ‘rows.csv?accessType=DOWNLOAD’

rows.csv?accessType     [            <=>     ] 116.30M  7.04MB/s    in 17s     

2021-03-03 13:08:08 (6.68 MB/s) - ‘rows.csv?accessType=DOWNLOAD’ saved [121946556]



/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (10,11,20,21,22,23,24,25) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
geolocator = Nominatim(user_agent="https://nominatim.openstreetmap.org/search?")
df=crime_df[crime_df['latitude'].isna()].copy()
lon_list=[]
lat_list=[]
for (i,j) in enumerate(df['address_1']):
    j=str(j)
    if "Block" in str(j):
        string = j.replace('Block',"") +" Buffalo New York"
    elif "&" in str(j):
        string = j[j.find("&")+2:]+" Buffalo New York"
    else :
        string = j+" Buffalo New York"
    try :
        location = geolocator.geocode(string)
        lat_list.append(location.latitude)
        lon_list.append(location.longitude)
    except :
        lat_list.append(None)
        lon_list.append(None)

In [4]:
df['latitude']=lat_list
df['longitude']=lon_list

In [5]:
crime_df = pd.concat([crime_df[crime_df['latitude'].notna()],df],axis=0)

In [6]:
crime_df.tail(15)

,incident_id,case_number,incident_datetime,incident_type_primary,incident_description,clearance_type,address_1,address_2,city,state,zip,country,latitude,longitude,created_at,updated_at,location,hour_of_day,day_of_week,parent_incident_type,Census Tract 1,Census Block 1,Census Block Group 1,Neighborhood 1,Police District 1,Council District 1,neighborhood,police_dt,council_dt
10607,NaN,21-0450312,2021-02-12 04:57:00,LARCENY/THEFT,Buffalo Police are investigating this report o...,NaN,300 Block HOLDEN,NaN,Buffalo,NY,NaN,NaN,NaN,NaN,02/14/2021 12:24:51 PM,NaN,NaN,12,SUNDAY,Theft,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10594,NaN,21-0440664,2021-02-13 01:00:00,UUV,Buffalo Police are investigating this report o...,NaN,0 Block MINERAL SPRINGS RD,NaN,Buffalo,NY,NaN,NaN,42.860985,-78.802705,02/13/2021 07:00:52 PM,NaN,NaN,19,SATURDAY,Theft of Vehicle,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10605,NaN,21-0450260,2021-02-14 11:01:57,LARCENY/THEFT,Buffalo Police are investigating this report o...,NaN,200 Block W FERRY ST,NaN,Buffalo,NY,NaN,NaN,42.915396,-78.894425,02/14/2021 11:15:00 AM,NaN,NaN,11,SUNDAY,Theft,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10701,NaN,21-0480189,2021-02-17 04:47:00,LARCENY/THEFT,Buffalo Police are investigating this report o...,NaN,500 Block NIAGARA ST,NaN,Buffalo,NY,NaN,NaN,42.897408,-78.888869,02/17/2021 09:02:38 AM,NaN,NaN,9,WEDNESDAY,Theft,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10982,NaN,21-0530456,2021-02-22 13:25:00,LARCENY/THEFT,Buffalo Police are investigating this report o...,NaN,200 Block W FERRY ST,NaN,Buffalo,NY,NaN,NaN,42.915396,-78.894425,02/22/2021 01:30:27 PM,NaN,NaN,13,MONDAY,Theft,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10838,NaN,21-0540261,2021-02-22 16:00:00,LARCENY/THEFT,Buffalo Police are investigating this report o...,NaN,1100 Block MAIN ST,NaN,Buffalo,NY,NaN,NaN,42.903729,-78.869246,02/23/2021 09:54:00 AM,NaN,NaN,9,TUESDAY,Theft,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10855,NaN,21-0540666,2021-02-23 18:08:15,UUV,Buffalo Police are investigating this report o...,NaN,1000 Block ELMWOOD AV,NaN,Buffalo,NY,NaN,NaN,42.924307,-78.877176,02/23/2021 06:09:15 PM,NaN,NaN,18,TUESDAY,Theft of Vehicle,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10868,NaN,21-0550100,2021-02-24 04:39:10,LARCENY/THEFT,Buffalo Police are investigating this report o...,NaN,ELMWOOD AV & AUBURN AV,NaN,Buffalo,NY,NaN,NaN,42.918639,-78.872281,02/24/2021 04:40:10 AM,NaN,NaN,4,WEDNESDAY,Theft,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10876,NaN,21-0550421,2021-02-24 12:37:00,LARCENY/THEFT,Buffalo Police are investigating this report o...,NaN,500 Block NIAGARA ST,NaN,Buffalo,NY,NaN,NaN,42.897408,-78.888869,02/24/2021 01:22:00 PM,NaN,NaN,13,WEDNESDAY,Theft,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1100,NaN,21-0580410,2021-02-25 12:45:00,LARCENY/THEFT,Buffalo Police are investigating this report o...,NaN,1900 Block S PARK AV,NaN,Buffalo,NY,NaN,NaN,42.847090,-78.823603,02/27/2021 12:26:20 PM,NaN,NaN,12,SATURDAY,Theft,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
crime_df.shape

(268335, 29)

In [8]:
crime_df.isna().sum()

incident_id               13209
case_number                   0
incident_datetime             5
incident_type_primary         0
incident_description          0
clearance_type           268335
address_1                    39
address_2                268335
city                          0
state                         0
zip                      241985
country                  234046
latitude                     68
longitude                    68
created_at                    0
updated_at                13209
location                   2044
hour_of_day                   0
day_of_week                   5
parent_incident_type          0
Census Tract 1           255290
Census Block 1           255290
Census Block Group 1     255290
Neighborhood 1           255290
Police District 1        255290
Council District 1       255316
neighborhood             268335
police_dt                268335
council_dt               268335
dtype: int64

In [9]:
#Reading the geojson data provided by https://data.buffalony.gov "https://data.buffalony.gov/Economic-Neighborhood-Development/Neighborhoods/q9bk-zu3p" :
nbh_polygon = gpd.read_file("/content/Neighborhoods.geojson")
pd_polygon = gpd.read_file("/content/BPD Districts.geojson")
cd_polygon = gpd.read_file("/content/Council Districts.geojson")

# Converting the pandas dataframe into a geopandas dataframe(has higher support for geospatial data) :
gdf = gpd.GeoDataFrame( crime_df, geometry=gpd.points_from_xy(crime_df.longitude, crime_df.latitude))

In [10]:
pd_polygon.head()

,bdy_left,name,st_prefix,rhs,st_code,hi_x_name,city_left,fromleft,lo_x_pre,direction,objectid,globalid,lhs,fromright,bdy_right,st_type,hi_x_type,lo_x_name,toright,lo_x_type,city_right,st_suffix,hi_x_suf,street,recnum_r,toleft,shape_stlength,lo_x_suf,shape_starea,recnum_l,hi_x_pre,sbe4_id,geometry
0,None,District B,None,None,None,None,None,0,None,None,3,{F6B399B4-524B-45CF-83A0-06C06BCCA628},None,0,None,None,None,None,0,None,None,None,None,None,0,0,62635.398738778073,None,152849720.51699948,0,None,0,"MULTIPOLYGON (((-78.90197 42.89666, -78.90113 ..."
1,None,District E,None,None,None,None,None,0,None,None,2,{83F2A179-549E-40B5-ACC4-5A0ED9DBF195},None,0,None,None,None,None,0,None,None,None,None,None,0,0,71419.982740536478,None,188176251.96103966,0,None,1,"MULTIPOLYGON (((-78.83168 42.94597, -78.83023 ..."
2,None,District C,None,None,None,None,None,0,None,None,4,{323FC3F5-857C-4E2E-A95D-4A8CC6112099},None,0,None,None,None,None,0,None,None,None,None,None,0,0,58574.689856890174,None,178912266.02432826,0,None,0,"MULTIPOLYGON (((-78.83762 42.87883, -78.83794 ..."
3,None,District A,None,None,None,None,None,0,None,None,1,{FF525ECB-B916-4644-9F18-A71150F86DAD},None,0,None,None,None,None,0,None,None,None,None,None,0,0,121348.34929364058,None,347940851.73374736,0,None,0,"MULTIPOLYGON (((-78.83838 42.83207, -78.85385 ..."
4,None,District D,None,None,None,None,None,0,None,None,5,{AB3A0367-7B20-4827-B06F-B41EC47B998B},None,0,None,None,None,None,0,None,None,None,None,None,0,0,89540.395914954919,None,274432466.58708191,0,None,0,"MULTIPOLYGON (((-78.90172 42.91518, -78.90168 ..."


In [11]:
#For loop going through all the neighborhoods, checking if each coordinate is in that neighborhood and assigning a boolean:
for i in range(len(nbh_polygon)):
# mask looks at the polygon geometry of each neighborhood and maps it :
    mask =  (nbh_polygon.loc[i, 'geometry'])
# This line checks if the coordinates in crime_df are inside the mask  and outputs a boolean list:
    pip_mask_geofence = gdf.within(mask)
    gdf['geofence'] = pip_mask_geofence
# Replace all values of the new column 'neighborhood'(not neighborhood_1) with the respective neighborhood name from polygon:
    gdf.loc[gdf['geofence'] == True, 'neighborhood'] = nbh_polygon['nbhdname'][i]


gdf.sort_values('incident_datetime',inplace=True)
gdf.drop('geofence',axis=1,inplace=True)
#gdf.drop('geometry',axis=1,inplace=True)

In [12]:
#For loop going through all the neighborhoods, checking if each coordinate is in that neighborhood and assigning a boolean:
for i in range(len(cd_polygon)):
# mask looks at the polygon geometry of each neighborhood and maps it :
    mask =  (cd_polygon.loc[i, 'geometry'])
# This line checks if the coordinates in crime_df are inside the mask  and outputs a boolean list:
    pip_mask_geofence = gdf.within(mask)
    gdf['geofence'] = pip_mask_geofence
# Replace all values of the new column 'neighborhood'(not neighborhood_1) with the respective neighborhood name from polygon:
    gdf.loc[gdf['geofence'] == True, 'council_dt'] = cd_polygon['councildistrict'][i]


gdf.sort_values('incident_datetime',inplace=True)
gdf.drop('geofence',axis=1,inplace=True)
#gdf.drop('geometry',axis=1,inplace=True)

In [13]:
#For loop going through all the neighborhoods, checking if each coordinate is in that neighborhood and assigning a boolean:
for i in range(len(pd_polygon)):
# mask looks at the polygon geometry of each neighborhood and maps it :
    mask =  (pd_polygon.loc[i, 'geometry'])
# This line checks if the coordinates in crime_df are inside the mask  and outputs a boolean list:
    pip_mask_geofence = gdf.within(mask)
    gdf['geofence'] = pip_mask_geofence
# Replace all values of the new column 'neighborhood'(not neighborhood_1) with the respective neighborhood name from polygon:
    gdf.loc[gdf['geofence'] == True, 'police_dt'] = pd_polygon['name'][i]


gdf.sort_values('incident_datetime',inplace=True)
gdf.drop('geofence',axis=1,inplace=True)
gdf.drop('geometry',axis=1,inplace=True)

In [14]:
#I removed the last 5 rows as they didnt have coordinates either:
gdf[-50:-5]

,incident_id,case_number,incident_datetime,incident_type_primary,incident_description,clearance_type,address_1,address_2,city,state,zip,country,latitude,longitude,created_at,updated_at,location,hour_of_day,day_of_week,parent_incident_type,Census Tract 1,Census Block 1,Census Block Group 1,Neighborhood 1,Police District 1,Council District 1,neighborhood,police_dt,council_dt
10972,NaN,21-0580338,2021-02-27 10:15:00,UUV,Buffalo Police are investigating this report o...,NaN,100 Block E DELAVAN AV,NaN,Buffalo,NY,NaN,NaN,42.922000,-78.850000,02/27/2021 10:46:11 AM,NaN,POINT (-78.85 42.922),10,SATURDAY,Theft of Vehicle,52.02,1007,1,Hamlin Park,District E,MASTEN,Hamlin Park,District E,MASTEN
11032,NaN,21-0590346,2021-02-27 11:52:00,LARCENY/THEFT,Buffalo Police are investigating this report o...,NaN,700 Block MAIN ST,NaN,Buffalo,NY,NaN,NaN,42.895000,-78.871000,02/28/2021 11:53:06 AM,NaN,POINT (-78.871 42.895),11,SUNDAY,Theft,25.02,1013,1,Fruit Belt,District B,ELLICOTT,Fruit Belt,District B,ELLICOTT
11042,NaN,21-0590369,2021-02-27 12:24:26,ASSAULT,Buffalo Police are investigating this report o...,NaN,400 Block GRIDER ST,NaN,Buffalo,NY,NaN,NaN,42.918000,-78.829000,02/28/2021 12:24:26 PM,NaN,POINT (-78.829 42.918),12,SUNDAY,Assault,34,5019,5,Delavan Grider,District E,MASTEN,Delavan Grider,District E,MASTEN
10974,NaN,21-0580450,2021-02-27 13:12:00,LARCENY/THEFT,Buffalo Police are investigating this report o...,NaN,800 Block PROSPECT AV,NaN,Buffalo,NY,NaN,NaN,42.908000,-78.897000,02/27/2021 01:13:00 PM,NaN,POINT (-78.897 42.908),13,SATURDAY,Theft,70,1010,1,West Side,District B,NIAGARA,West Side,District B,NIAGARA
10979,NaN,21-0580545,2021-02-27 15:13:00,ROBBERY,Buffalo Police are investigating this report o...,NaN,200 Block ONTARIO ST,NaN,Buffalo,NY,NaN,NaN,42.951000,-78.902000,02/27/2021 03:20:00 PM,NaN,POINT (-78.902 42.951),15,SATURDAY,Robbery,57,3008,3,Riverside,District D,NORTH,Riverside,District D,NORTH
10980,NaN,21-0580556,2021-02-27 15:30:15,ASSAULT,Buffalo Police are investigating this report o...,NaN,400 Block DELAWARE AV,NaN,Buffalo,NY,NaN,NaN,42.895000,-78.875000,02/27/2021 03:30:15 PM,NaN,POINT (-78.875 42.895),15,SATURDAY,Assault,68,2011,2,Allentown,District B,FILLMORE,Allentown,District B,FILLMORE
10983,NaN,21-0580566,2021-02-27 15:41:00,LARCENY/THEFT,Buffalo Police are investigating this report o...,NaN,400 Block GRANT ST,NaN,Buffalo,NY,NaN,NaN,42.927000,-78.890000,02/27/2021 04:22:00 PM,NaN,POINT (-78.89 42.927),16,SATURDAY,Theft,171,2002,2,Upper West Side,District D,NORTH,Upper West Side,District D,NORTH
10984,NaN,21-0580586,2021-02-27 16:00:00,ASSAULT,Buffalo Police are investigating this report o...,NaN,300 Block TONAWANDA ST,NaN,Buffalo,NY,NaN,NaN,42.947000,-78.902000,02/27/2021 04:50:00 PM,NaN,POINT (-78.902 42.947),16,SATURDAY,Assault,57,2003,2,Riverside,District D,NORTH,Riverside,District D,NORTH
10986,NaN,21-0580609,2021-02-27 16:26:52,ASSAULT,Buffalo Police are investigating this report o...,NaN,100 Block SOUTHSIDE PW,NaN,Buffalo,NY,NaN,NaN,42.853000,-78.823000,02/27/2021 04:26:52 PM,NaN,POINT (-78.823 42.853),16,SATURDAY,Assault,8,2001,2,South Park,District A,SOUTH,South Park,District A,SOUTH
10987,NaN,21-0580645,2021-02-27 17:03:00,ASSAULT,Buffalo Police are investigating this report o...,NaN,0 Block ZITTEL ST,NaN,Buffalo,NY,NaN,NaN,42.856000,-78.806000,02/27/2021 05:04:15 PM,NaN,POINT (-78.806 42.856),17,SATURDAY,Assault,10,5011,5,Seneca-Cazenovia,District A,SOUTH,Seneca-Cazenovia,District A,SOUTH


In [15]:
# checking null values:
gdf.isna().sum()

incident_id               13209
case_number                   0
incident_datetime             5
incident_type_primary         0
incident_description          0
clearance_type           268335
address_1                    39
address_2                268335
city                          0
state                         0
zip                      241985
country                  234046
latitude                     68
longitude                    68
created_at                    0
updated_at                13209
location                   2044
hour_of_day                   0
day_of_week                   5
parent_incident_type          0
Census Tract 1           255290
Census Block 1           255290
Census Block Group 1     255290
Neighborhood 1           255290
Police District 1        255290
Council District 1       255316
neighborhood               2929
police_dt                  2891
council_dt                 2968
dtype: int64

In [16]:
gdf.head()

,incident_id,case_number,incident_datetime,incident_type_primary,incident_description,clearance_type,address_1,address_2,city,state,zip,country,latitude,longitude,created_at,updated_at,location,hour_of_day,day_of_week,parent_incident_type,Census Tract 1,Census Block 1,Census Block Group 1,Neighborhood 1,Police District 1,Council District 1,neighborhood,police_dt,council_dt
59934,55236827.0,11-0050386,1910-12-22 00:00:00,LARCENY/THEFT,Buffalo Police are investigating this report o...,NaN,200 Block CRESTWOOD AV,NaN,BUFFALO,NY,NaN,NaN,42.952764,-78.859972,01/06/2011 07:00:48 AM,10/10/2019 07:22:02 PM,POINT (-78.85997246003562 42.952764317373884),0,Thursday,Theft,NaN,NaN,NaN,NaN,NaN,NaN,North Park,District D,DELAWARE
121847,710086379.0,15-0760407,1914-12-01 22:00:00,LARCENY/THEFT,Buffalo Police are investigating this report o...,NaN,100 Block LINCOLN PW,NaN,BUFFALO,NY,NaN,NaN,42.931105,-78.874554,03/18/2015 06:06:31 AM,09/23/2019 11:22:56 PM,POINT (-78.87455447748493 42.93110482058636),22,Tuesday,Theft,NaN,NaN,NaN,NaN,NaN,NaN,Elmwood Bidwell,District D,DELAWARE
205089,942663542.0,06-1840984,1951-07-03 19:51:00,UUV,Buffalo Police are investigating this report o...,NaN,200 Block W FERRY ST,NaN,BUFFALO,NY,NaN,NaN,42.915005,-78.892808,09/24/2019 10:49:30 PM,09/24/2019 10:56:37 PM,POINT (-78.892808174024 42.915005299524026),19,Tuesday,Theft of Vehicle,NaN,NaN,NaN,NaN,NaN,NaN,Upper West Side,District B,NIAGARA
147150,146621859.0,13-0730379,1951-12-05 02:20:21,ASSAULT,Buffalo Police are investigating this report o...,NaN,200 Block CAMBRIDGE AV,NaN,BUFFALO,NY,NaN,NaN,42.921123,-78.822961,03/15/2013 06:00:48 AM,03/21/2013 06:00:42 AM,POINT (-78.822961 42.921123),2,Wednesday,Assault,NaN,NaN,NaN,NaN,NaN,NaN,Genesee-Moselle,District E,MASTEN
234055,942606470.0,08-3530528,1952-08-30 16:00:00,LARCENY/THEFT,Buffalo Police are investigating this report o...,NaN,200 Block JEFFERSON AV,NaN,BUFFALO,NY,NaN,NaN,42.886733,-78.853599,09/24/2019 10:32:01 PM,09/24/2019 10:34:37 PM,POINT (-78.85359911134378 42.88673286341301),16,Saturday,Theft,NaN,NaN,NaN,NaN,NaN,NaN,Broadway Fillmore,District C,ELLICOTT


In [17]:
gdf.to_csv('crime.csv')